# Projet 4

In [58]:
import requests
from bs4 import BeautifulSoup

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [59]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    prices = {}
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    tag = soup.find('div',attrs={'class': 'pricing-table'})
    
    for tag in soup.findAll('div',attrs={'class': 'pricing-table'}) :
        entity = tag.find('h2').text
        price = tag.find('span',{'class':"pricing-table-price"}).text.split()[0]
        storage = tag.find_all('li')[3].text.split()[0]
        databases = tag.find_all('li')[4].text.split()[0]
    
        prices[entity] = {'price':price, 'storage':storage,'databases':databases}

    return prices

get_prices_from_url(URL_PAGE2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': '1'},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': '5'},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': '25'}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [60]:
def extract_beer_infos(url):
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    name   = soup.find('div', {'class' : "product-detail-info-title"}).find('h1').text
    note   = int(soup.find('div',{'class':"stars"})['data-percent'])
    price  = float(soup.find('span',{'class':"price"}).text.split()[0].replace(',','.'))
    volume = int(soup.find('dd',{'class':"small-6 medium-9 columns js-beer-volume"})['data-volume'].split()[0])
    
    infos = {
        'name': name,
        'note': note,
        'price': price,
        'volume': volume,
    }
    return infos

extract_beer_infos("https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33")

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [61]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [62]:
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    
    data = requests.get(url).json
    pages = ["https://www.beerwulf.com"+item['contentReference'] for item in data['items']]
    
    # Sequential version (slow):
    beers = []
    for product in pages:
        beers.append(extract_beer_infos(product))
    

    # Parallel version (faster):
    # beers = []
    
    return beers

extract_beer_list_infos(URL_BEERLIST_FRANCE)

TypeError: 'method' object is not subscriptable

In [65]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [66]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... FAIL
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... FAIL
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ERROR

ERROR: test_03_extract_beer_list_infos (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-65-fbabfd96df78>", line 31, in test_03_extract_beer_list_infos
    infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
  File "<ipython-input-62-675bdd13c821>", line 5, in extract_beer_list_infos
    pages = ["https://www.beerwulf.com"+item['contentReference'] for item in data['items']]
TypeError: 'method' object is not subscriptable

FAIL: test_01_get_prices_from_url_page2 (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-65-fbabfd96df78>", line 19, in test_01_get_prices_from_url_page2
    self.as